In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p polars,instructor,openai,ollama

Author: Prayson W. Daniel

Last updated: 2024-10-31T12:36:06.383355+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

polars    : 1.7.1
instructor: 1.4.3
openai    : 1.46.1
ollama    : 0.3.3

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
from typing import Final, Literal

import instructor
import polars as pl
from openai import OpenAI
from pydantic import BaseModel, Field

In [3]:
MODEL: Final = "gemma2"

VALID_EMOTIONS: tuple[str, ...] = tuple(
    {
        "admiration",
        "approval",
        "gratitude",
        "optimism",
        "joy",
        "neutral",
        "disapproval",
        "disappointment",
        "love",
        "relief",
        "excitement",
        "desire",
        "caring",
        "pride",
        "realization",
        "disgust",
        "annoyance",
        "remorse",
        "amusement",
        "anger",
        "grief",
        "curiosity",
        "fear",
        "surprise",
        "sadness",
        "embarrassment",
        "confusion",
        "nervousness",
    }
)

In [4]:
class Emotion(BaseModel):
    kind: Literal[VALID_EMOTIONS]  # type: ignore # literal does not like dynamic values
    reason: str = Field(
        ..., description="A short third person reason for selected the emotion"
    )

In [5]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)

In [6]:
def classifier(text: str, model: str = MODEL) -> Emotion:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": ("You are a brilliant emotion pyschologist."
                            f"What is the text emotion: Text: {text}"),
            }
        ],
        response_model=Emotion,
        temperature=0.0,
    )

    return response

In [7]:
pl.Config.set_fmt_str_lengths(100).set_tbl_rows(
    -1
)  # show more text. default is 25, show all rowss

dataf = pl.read_json("../data/examples.json").explode(["emotion", "text"])

In [8]:
dataf.head(3)

emotion,text
str,str
"""admiration""","""The way she solved that complex math problem was incredible. Truly a mind to admire!"""
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!"""
"""admiration""","""Her dedication to helping the homeless inspires me every day."""


In [9]:
classifier("I dont' know how to think about where the would is going?")

Emotion(kind='confusion', reason='The speaker expresses uncertainty and lack of direction regarding the future.')

In [ ]:
(
    dataf.sample(8)
    .with_columns(
        pl.col("text")
        .map_elements(classifier, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.kind, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reason, return_dtype=pl.String)
        .alias("reason"),
    )
)